## Purpose
This notebook imports images and their labels from the ai_club team folder and exports the images, cropped to their ROI. Then it creates synthetic data by stretching both the left and right vocal cords.


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imagesize
from PIL import Image, ImageFilter

In [ ]:
# define input/output folders
source_folder = '/data/ai_club/team_13_2024-25/VIPR/Data'
images_folder = os.path.join(source_folder,'YOLO_images_total')
labels_folder = os.path.join(source_folder,'YOLO_labels_total')
output_folder = 'ROI_cropped_images'

In [ ]:
image_list = os.listdir(images_folder)

image_path_list = [os.path.join(images_folder,image) for image in os.listdir(images_folder)]
label_path_list = [os.path.join(labels_folder,label) for label in os.listdir(labels_folder)]

In [ ]:
def blend_halves_bilinear(left_img, right_img, blend_width=10):
    """
    Combine left_img and right_img without removing any pixels in the overlapping region.
    The overlapping seam is defined as the rightmost blend_width columns of left_img and
    the leftmost blend_width columns of right_img. These two regions are first resampled
    using bilinear interpolation to a common width (2*blend_width), then blended together
    with a 2D alpha mask that transitions smoothly from 0 (left image) to 1 (right image).
    """
    width_left, height = left_img.size
    width_right, _ = right_img.size

    # Create composite image: paste left_img fully, then right_img fully shifted.
    total_width = width_left + width_right
    composite = Image.new(left_img.mode, (total_width, height))
    composite.paste(left_img, (0, 0))
    composite.paste(right_img, (width_left, 0))

    # Define the seam region in the composite:
    seam_start = width_left - blend_width  # start of overlap in composite
    seam_width = 2 * blend_width            # full width of overlap region

    # Extract the overlapping regions:
    left_overlap = left_img.crop((width_left - blend_width, 0, width_left, height))
    right_overlap = right_img.crop((0, 0, blend_width, height))

    # Resample each overlap to a common region of width = 2*blend_width using bilinear interpolation.
    left_resized = left_overlap.resize((seam_width, height), resample=Image.Resampling.BILINEAR)
    right_resized = right_overlap.resize((seam_width, height), resample=Image.Resampling.BILINEAR)

    # Convert to numpy arrays (float32 for interpolation math)
    left_arr = np.array(left_resized, dtype=np.float32)
    right_arr = np.array(right_resized, dtype=np.float32)

    # Create a 2D alpha mask for blending.
    # Here the mask is purely horizontal—every row gets the same gradient from 0 (left) to 1 (right).
    # (If desired you could incorporate a vertical component too.)
    alpha = np.tile(np.linspace(0, 1, seam_width), (height, 1))
    
    # Blend the two resampled overlap regions using the alpha mask.
    # For each pixel: blended = (1 - alpha) * left_pixel + alpha * right_pixel.
    blended_region = (1 - alpha)[..., None] * left_arr + alpha[..., None] * right_arr

    # Convert back to a PIL Image.
    blended_region_img = Image.fromarray(np.clip(blended_region, 0, 255).astype(np.uint8), mode=left_img.mode)
    
    # Paste the blended seam back into the composite image.
    composite.paste(blended_region_img, (seam_start, 0))

    return composite

In [ ]:
def crop_and_stretch(image_path, label_path, scaleFactor=1.2, blend_width=10):
    """Crops a 256x256 image to the region of interest (ROI) specified by the label file and resizes it back to 256x256."""
    # Verify that the image and label file have matching base names.
    image_base = os.path.splitext(os.path.basename(image_path))[0]
    label_base = os.path.splitext(os.path.basename(label_path))[0]
    if image_base != label_base:
        raise ValueError("Image file and label file names do not match.")

    # Open the image.
    image = Image.open(image_path)
    if image.size != (256, 256):
        raise ValueError("The input image is not 256x256 in size.")

    # Read and parse the label file.
    with open(label_path, 'r') as f:
        line = f.readline().strip()
        parts = line.split()
        if len(parts) != 5:
            raise ValueError("Label file does not contain exactly 5 values.")
        # Parse values: ignoring the first value which represents the class label.
        try:
            _, x_center, y_center, box_width, box_height = parts
            x_center = float(x_center)
            y_center = float(y_center)
            box_width = float(box_width)
            box_height = float(box_height)
        except ValueError:
            raise ValueError("One or more of the coordinate values are not valid floats.")

    # Calculate pixel coordinates for cropping.
    img_width, img_height = image.size
    left   = int((x_center - box_width / 2) * img_width)
    top    = int((y_center - box_height / 2) * img_height)
    right  = int((x_center + box_width / 2) * img_width)
    bottom = int((y_center + box_height / 2) * img_height)
    bottom_stretched = int((y_center + (box_height*scaleFactor) / 2) * img_height)
    mid = int(x_center * img_width)
    
    # Ensure coordinates are within image boundaries.
    left = max(0, left)
    top = max(0, top)
    right = min(img_width, right)
    bottom = min(img_height, bottom)
    bottom_stretched = min(img_height, bottom_stretched)
    stretched_height = bottom_stretched - top

    # Crop the image to the region of interest.
    cropped_image = image.crop((left, top, right, bottom))
    
    # Split image into left and right halves
    left_half = image.crop((left,top,mid,bottom))
    right_half = image.crop((mid,top,right,bottom))
    
    # Create extended halves
    left_half_ext = image.crop((left,top,mid,bottom_stretched))
    right_half_ext = image.crop((mid,top,right,bottom_stretched))
    
    # Create "stretched" halves
    left_half_str = left_half.resize((left_half.size[0],stretched_height), Image.Resampling.LANCZOS)
    right_half_str = right_half.resize((right_half.size[0],stretched_height), Image.Resampling.LANCZOS)
    
    # Resize healthy cropped image back to 256x256 using a high-quality filter.
    healthy = cropped_image.resize((256, 256), Image.Resampling.LANCZOS)
    
    # Create alternate healthy image with bilinear interpolation artifact
    healthy2 = blend_halves_bilinear(left_half, right_half, blend_width)
    healthy2 = healthy2.resize((256, 256), Image.Resampling.LANCZOS)

    # Reconstruct left paralyzed image
    left_par = blend_halves_bilinear(left_half_str, right_half_ext, blend_width)
    left_par = left_par.resize((256, 256), Image.Resampling.LANCZOS)
    
    # Reconstruct right paralyzed image
    right_par = blend_halves_bilinear(left_half_ext, right_half_str, blend_width)
    right_par = right_par.resize((256, 256), Image.Resampling.LANCZOS)
    

    return healthy, healthy2, left_par, right_par

In [ ]:
# Set parameters
i = 64
scaleFactor = 1.5
blend_width = 2

# Generate sample images
image = image_list[i]
label_split = image.split('.')
label = os.path.join(source_folder, labels_folder, str(label_split[0]+'.'+label_split[1]+'.txt'))
[healthy, healthy2, left_par, right_par] = crop_and_stretch(os.path.join(images_folder,image), label, scaleFactor, blend_width)

# Display output
healthy.show()
healthy2.show()
left_par.show()
right_par.show()

## Save All Images:

In [ ]:
scaleFactor = 1.5
blend_width = 2
for i in range(len(image_list)):
    print('saving batch ',i,' of images')
    image = image_list[i]
    label_split = image.split('.')
    label = os.path.join(source_folder, labels_folder, str(label_split[0]+'.'+label_split[1]+'.txt'))
    print(image)
    print(label)
    [healthy, healthy2, left_par, right_par] = crop_and_stretch(os.path.join(images_folder,image), label, scaleFactor, blend_width)
    
#     print(os.path.join(source_folder, output_folder, 'leftpar_'+str(image)))
    left_par.save(os.path.join(source_folder, output_folder, 'leftpar_'+str(image)), format="PNG")
    right_par.save(os.path.join(source_folder, output_folder, 'rightpar_'+str(image)), format="PNG")
    healthy.save(os.path.join(source_folder, output_folder, 'healthy_'+str(image)), format="PNG")
    healthy2.save(os.path.join(source_folder, output_folder, 'healthy2_'+str(image)), format="PNG")
    print('batch saved succesfully')